# NOTE:  If this was installed from my Docker image, then skip to Step 1: Imports

I built this Docker image because of the issues people were having getting this to run.  Some problems and changes made:
- tokenizers version
- transformers version
- msticpy version
- torch version + GPU
- Utils package needed to be modified to use GPU 0 instead of the hardcoded GPU 1
- The build.sh script takes care of the file structure
- The run.sh script sets up and runs your container

# MitreMap - Infer MITRE technique from Threat Intel Data

__Notebook Version:__ 1.0 <br>
__Notebook Author:__ Vani Asawa<br>


__Python Version:__ >=Python 3.8<br>
__Platforms Supported:__  Azure Machine Learning Notebooks<br>

__Data Source Required:__ None<br>

__GPU Compute Required:__ No<br>
__GPU Compute Recommended:__ Yes<br>

__Requirements Path:__ ```../mitremap-notebook/requirements.txt```<br>
__Packages Downloaded:__ 
- ipywidgets==7.5.1
- tokenizers==0.10.3
- transformers==4.6.0
- msticpy==2.2.0
- nltk==3.6.2
- iocextract==1.13.1
- shap==0.41.0

## Overview

This notebook allows a user to map descriptive text of an incident on to relevant MITRE ATT&CK Enterprise techniques. It uses a [GPT2](https://huggingface.co/gpt2) language model to associate terms in the description with similar descriptions in past incidents. It also extracts relevant Indicators of Compromise from the text.

You can use the notebook with one of several pre-trained models or train your own model using your own threat reports or public sources.

## Motivation

Please refer to [Motivation](./README.md#motivation) and [Goals](./README.md#goals-of-the-mitremap-notebook) to learn more.

## Prerequisites
**Please do not run the notebook cells all at once**. The cells need to be run sequentially and successfully executed before proceeding with the remainder of the notebook.

## Table of Contents

0. Installations [One-Time Setup]
1. Imports
2. Configure Input Data and Model Parameters
3. Run
4. Results

## 0. Installations [One-Time Setup]

Please refer to [One-Time Setup](./README.md#one-time-setup) to configure the virtual environment, install the required packages, and download the model artifacts.

Use the Powershell or BASH script below to download the model artifacts 

**Estimated Time to download the model artifacts** - 5-10 minutes

### Option 1: Powershell

In [1]:
!PowerShell ./model.ps1

./model.ps1 : File D:\git\Azure-Sentinel-Notebooks\mitremap-notebook\model.ps1 cannot be loaded because running 
scripts is disabled on this system. For more information, see about_Execution_Policies at 
https:/go.microsoft.com/fwlink/?LinkID=135170.
At line:1 char:1
+ ./model.ps1
+ ~~~~~~~~~~~
    + CategoryInfo          : SecurityError: (:) [], PSSecurityException
    + FullyQualifiedErrorId : UnauthorizedAccess


### Option 2: BASH

In [ ]:
%%bash
./model.sh

**Download the utils whl to use the inference packages**

In [2]:
%pip install utils-1.0-py3-none-any.whl

Processing d:\git\azure-sentinel-notebooks\mitremap-notebook\utils-1.0-py3-none-any.whl
Note: you may need to restart the kernel to use updated packages.


Re-start the kernel and run the Notebook from **1. Imports**.

##  1. Imports

The modules used to run this notebook can be found under ```mitremap-notebook/utils/*```

In [1]:
import os
import sys
sys.path.append(os.getcwd())

import utils
from utils import main, inference, configs

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


## 2. Configure Input Data and Model Parameters,

Please refer to [Input Parameters](./README.md#input-parameters) to learn more about setting the input parameter configurations.

Start using the notebook with one of the threat intel examples in the markdown script 😊

### Input Parameters
IMPORTANT In order to view the widgets in your Notebook, you must download the following jupyter extension via Terminal - jupyter labextension install @jupyter-widgets/jupyterlab-manager

The notebook requires the following parameters from the user:

Threat Intel Data: -
Unstructured, English threat report that the user would like to process through the NLP model.

The notebook will chunk the threat intel report into batches of 3 sentences, and apply the model on each batch to get the corresponding MITRE Enterprise Technique, to prevent any loss of information.

Sample reports:

#### 1
Adversaries may abuse msiexec.exe to proxy execution of malicious payloads. Msiexec.exe is the command-line utility for the Windows Installer and is thus commonly associated with executing installation packages (.msi). The Msiexec.exe binary may also be digitally signed by Microsoft.
#### 2
This query over Azure Active Directory sign-in considers all user sign-ins for each Azure Active Directory application and picks out the most anomalous change in location profile for a user within an individual application. The intent is to hunt for user account compromise, possibly via a specific applicationvector.
#### 3
Threat actors can use auditpol binary to change audit policy configuration to impair detection capability. This can be carried out by selectively disabling/removing certain audit policies as well as restoring a custom policy owned by the threat actor.
#### 4
When the trojan starts up it will attempt to install a scheduled task with the name of “Java Maintenance64” to keep itself running.
#### 5
Detects exploitation attempt against Citrix Netscaler, Application Delivery Controller (ADS) and Citrix Gateway exploiting vulnerabilities reported as CVE-2020-8193 and CVE-2020-8195
Confidence Score Threshold:
The TTP predictions for a sample TI input data have an associated confidence score from the NLP model, ranging from 0 (not very confident) to 1 (most confident).

Filter the results to predictions with confidence >= threshold configured by the user.

Default threshold: 0.6


Obtain Model Explainability:
Obtain further insights into which words and phrases in your input data contributed to the TTP prediction, using SHAP values.

Note: Model Explainability will increase the time taken to obtain the inference results for your notebook!

Default value: True

In [2]:
config_widgets = configs.configure_model_parameters()
for k in config_widgets.keys():
       display(config_widgets[k])

Textarea(value='', description='Threat Intel Report:', layout=Layout(height='200px', width='80%'), placeholder…

FloatSlider(value=0.6, description='Confidence Score Threshold: ', layout=Layout(height='30px', width='50%'), …

Select(description='Obtain Model Explainability?: ', layout=Layout(height='80px', width='50%'), options=('True…

## 3. Run

Time to run the ```main.go``` function depends on the -

1. Length of the Threat Intel Report, and
2. If **Model Explainability** is set to True

For our sample threat reports in the markdown script, you can expect -

- < 1 minute without model explainability, and 
- 1-2 minutes with model explainability.

In [3]:
configs, inference_df, iocs_df = main.go(
    config_widgets
)

#################### SUMMARY #################### 

Threat Intel (TI) Report: 
	'Adversaries may abuse msiexec.exe to proxy execution of malicious payloads. Msiexec.exe is the command-line utility for the Windows Installer and is thus commonly associated with executing installation packages (.msi). The Msiexec.exe binary may also be digitally signed by Microsoft.'

Confidence Score Threshold: 0.6

Obtain Model Explainability Summary: True

################################################# 

Tokenizer artifact obtained from path /notebook/distilgpt2-512/tokenizer
Labels artifact obtained from path /notebook/distilgpt2-512/labels


Downloading:   0%|          | 0.00/762 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/353M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilgpt2 were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at distilgpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model artifact obtained from path /notebook/distilgpt2-512/model_state_dicts
Model on device 'cuda'


## 4. Results

- ```configs```: Stores the input configurations set by the customer

- ```inference_df```: Stores the inference results for the threat intel data

- ```iocs_df```: Stores the IOCs extracted from the threat intel data.

Use the ```inference.print_detailed_report(inference_df, configs)``` to obtain a printed summary of the MITRE technique predictions.

In [4]:
inference.print_detailed_report(
    inference_df,
    configs
)

Sentences # : 1-3 

Threat Intel Data: 
Adversaries may abuse msiexec.exe to proxy execution of malicious payloads. Msiexec.exe is the command-line utility for the Windows Installer and is thus commonly associated with executing installation packages (.msi). The Msiexec.exe binary may also be digitally signed by Microsoft.

Predicted MITRE Technique: 
{
	'technique': T1218, 
	'technique_name': System Binary Proxy Execution, 
	'webpage_link': https://attack.mitre.org/techniques/T1218/, 
	'confidence_score': 0.89854
}

Model Explainability: 
Shap Legend: 
{
	'Red': Positive Contribution to Predicted Label, 
	'Blue': Negative Contribution to Predicted Label
}

Shap Text Plot: 



Top 10 tokens with Positive SHAP Contribution to predicted MITRE TTP: 
[('Ad', 0.07838), ('vers', 0.0105), ('aries', 0.01188), ('may', 0.00075), ('.', 0.0182), ('exe', 0.19653), ('to', 0.01082), ('execution', 0.03843), ('is', 0.03241), ('the', 0.00216)]

Top 10 tokens with Negative SHAP Contribution to predicted MITRE TTP: 
[('abuse', -0.00573), ('ms', -0.00573), ('ie', -0.01432), ('x', -0.01432), ('ec', -0.01432), ('proxy', -0.00141), ('of', -0.001), ('malicious', -0.001), ('payload', -0.01094), ('s', -0.01094)]

**************************************************


In [5]:
print('Summary Statistics for Inference Dataframe: ')
print('Shape of Inference Dataframe: ', inference_df.shape)
if not inference_df.empty:
    print('Sample rows: ')
    display(inference_df.head(5))
else:
    print('No results obtained.')

Summary Statistics for Inference Dataframe: 
Shape of Inference Dataframe:  (1, 9)
Sample rows: 


,chunk_#,sentences_#,threat_intel,processed_threat_intel,output,model,iocs,shap_base,shap_contribution
0,1,1-3,Adversaries may abuse msiexec.exe to proxy exe...,adversary may abuse msiexec.exe proxy executio...,"{'label': 'LABEL_95', 'score': 0.8985441923141...",distilgpt2-512,"{'IP': [], 'EMAIL': [], 'URL': [], 'YARA': [],...","{'values': [0.07838, 0.0105, 0.01188, 0.00075,...","{'positive': {'Ad': 0.07838, 'vers': 0.0105, '..."


In [6]:
print('Summary Statistics for IOCs Dataframe: ')
print('Shape of IOCs Dataframe: ', iocs_df.shape)
if not iocs_df.empty:
    print('Distinct counts for each category of IOCs: ')
    display(iocs_df.groupby('IOC_Type').count().rename(columns={'IOC_Value': 'Count'}))
    print('Sample rows: ')
    display(iocs_df.head(5))
else:
    print('No IOCs obtained.')

Summary Statistics for IOCs Dataframe: 
Shape of IOCs Dataframe:  (0, 0)
No IOCs obtained.
